<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Ingrid", "transactions": [{"transaction-id": 41, "amount": -248}, {"transaction-id": 91, "amount": -192}, {"transaction-id": 156, "amount": -213}, {"transaction-id": 209, "amount": -10}, {"transaction-id": 215, "amount": -162}, {"transaction-id": 249, "amount": -184}, {"transaction-id": 300, "amount": -184}, {"transaction-id": 324, "amount": -297}, {"transaction-id": 624, "amount": -210}, {"transaction-id": 701, "amount": -138}, {"transaction-id": 726, "amount": -217}, {"transaction-id": 739, "amount": -196}, {"transaction-id": 756, "amount": -114}, {"transaction-id": 758, "amount": 50}, {"transaction-id": 783, "amount": -108}, {"transaction-id": 792, "amount": -265}, {"transaction-id": 915, "amount": -276}, {"transaction-id": 916, "amount": -101}, {"transaction-id": 1024, "amount": -248}, {"transaction-id": 1030, "amount": -378}, {"transaction-id": 1050, "amount": -247}, {"transaction-id": 1076, "amount": -289}, {"transaction-id": 1111, "amount": -274}, {"transact

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Ingrid", "transactions": [{"transaction-id": 41, "amount": -248}, {"transaction-id": 91, "amount": -192}, {"transaction-id": 156, "amount": -213}, {"transaction-id": 209, "amount": -10}, {"transaction-id": 215, "amount": -162}, {"transaction-id": 249, "amount": -184}, {"transaction-id": 300, "amount": -184}, {"transaction-id": 324, "amount": -297}, {"transaction-id": 624, "amount": -210}, {"transaction-id": 701, "amount": -138}, {"transaction-id": 726, "amount": -217}, {"transaction-id": 739, "amount": -196}, {"transaction-id": 756, "amount": -114}, {"transaction-id": 758, "amount": 50}, {"transaction-id": 783, "amount": -108}, {"transaction-id": 792, "amount": -265}, {"transaction-id": 915, "amount": -276}, {"transaction-id": 916, "amount": -101}, {"transaction-id": 1024, "amount": -248}, {"transaction-id": 1030, "amount": -378}, {"transaction-id": 1050, "amount": -247}, {"transaction-id": 1076, "amount": -289}, {"transaction-id": 1111, "amount": -274}, {"transact

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Ingrid',
  'transactions': [{'transaction-id': 41, 'amount': -248},
   {'transaction-id': 91, 'amount': -192},
   {'transaction-id': 156, 'amount': -213},
   {'transaction-id': 209, 'amount': -10},
   {'transaction-id': 215, 'amount': -162},
   {'transaction-id': 249, 'amount': -184},
   {'transaction-id': 300, 'amount': -184},
   {'transaction-id': 324, 'amount': -297},
   {'transaction-id': 624, 'amount': -210},
   {'transaction-id': 701, 'amount': -138},
   {'transaction-id': 726, 'amount': -217},
   {'transaction-id': 739, 'amount': -196},
   {'transaction-id': 756, 'amount': -114},
   {'transaction-id': 758, 'amount': 50},
   {'transaction-id': 783, 'amount': -108},
   {'transaction-id': 792, 'amount': -265},
   {'transaction-id': 915, 'amount': -276},
   {'transaction-id': 916, 'amount': -101},
   {'transaction-id': 1024, 'amount': -248},
   {'transaction-id': 1030, 'amount': -378},
   {'transaction-id': 1050, 'amount': -247},
   {'transaction-id': 1076, 'am

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 75,
  'name': 'Alice',
  'transactions': [{'transaction-id': 1035, 'amount': 749},
   {'transaction-id': 2797, 'amount': 726},
   {'transaction-id': 2945, 'amount': 780},
   {'transaction-id': 4270, 'amount': 854},
   {'transaction-id': 4817, 'amount': 640},
   {'transaction-id': 6011, 'amount': 697},
   {'transaction-id': 6058, 'amount': 782},
   {'transaction-id': 6314, 'amount': 827},
   {'transaction-id': 6808, 'amount': 814},
   {'transaction-id': 6912, 'amount': 776},
   {'transaction-id': 7341, 'amount': 794},
   {'transaction-id': 7356, 'amount': 736},
   {'transaction-id': 7768, 'amount': 805},
   {'transaction-id': 8076, 'amount': 728},
   {'transaction-id': 8723, 'amount': 754},
   {'transaction-id': 9058, 'amount': 790}]},
 {'id': 113,
  'name': 'Alice',
  'transactions': [{'transaction-id': 46, 'amount': 3127},
   {'transaction-id': 163, 'amount': 2932},
   {'transaction-id': 238, 'amount': 2943},
   {'transaction-id': 403, 'amount': 3071},
   {'transaction-id': 91

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 16},
 {'name': 'Alice', 'count': 22},
 {'name': 'Alice', 'count': 11},
 {'name': 'Alice', 'count': 23},
 {'name': 'Alice', 'count': 15})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(16, 22, 11, 23, 15)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

17.85333333333333

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 1035, 'amount': 749},
  {'transaction-id': 2797, 'amount': 726},
  {'transaction-id': 2945, 'amount': 780},
  {'transaction-id': 4270, 'amount': 854},
  {'transaction-id': 4817, 'amount': 640},
  {'transaction-id': 6011, 'amount': 697},
  {'transaction-id': 6058, 'amount': 782},
  {'transaction-id': 6314, 'amount': 827},
  {'transaction-id': 6808, 'amount': 814},
  {'transaction-id': 6912, 'amount': 776},
  {'transaction-id': 7341, 'amount': 794},
  {'transaction-id': 7356, 'amount': 736},
  {'transaction-id': 7768, 'amount': 805},
  {'transaction-id': 8076, 'amount': 728},
  {'transaction-id': 8723, 'amount': 754},
  {'transaction-id': 9058, 'amount': 790}],
 [{'transaction-id': 46, 'amount': 3127},
  {'transaction-id': 163, 'amount': 2932},
  {'transaction-id': 238, 'amount': 2943},
  {'transaction-id': 403, 'amount': 3071},
  {'transaction-id': 917, 'amount': 2922},
  {'transaction-id': 2893, 'amount': 3139},
  {'transaction-id': 3710, 'amount': 3093},
  {'trans

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 1035, 'amount': 749},
 {'transaction-id': 2797, 'amount': 726},
 {'transaction-id': 2945, 'amount': 780})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(749, 726, 780)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

814.8780806572069

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 72), ('Alice', 72), ('Alice', 78), ('Alice', 78), ('Bob', 122), ('Bob', 123), ('Bob', 131), ('Bob', 134), ('Charlie', 107), ('Charlie', 108), ('Charlie', 117), ('Charlie', 117), ('Dan', 126), ('Dan', 132), ('Dan', 137), ('Dan', 140), ('Edith', 133), ('Edith', 135), ('Edith', 147), ('Edith', 148), ('Frank', 204), ('Frank', 204), ('Frank', 221), ('Frank', 221), ('George', 24), ('George', 24), ('George', 26), ('George', 26), ('Hannah', 80), ('Hannah', 82), ('Hannah', 87), ('Hannah', 91), ('Ingrid', 78), ('Ingrid', 80), ('Ingrid', 85), ('Ingrid', 90), ('Jerry', 132), ('Jerry', 132), ('Jerry', 142), ('Jerry', 143), ('Kevin', 70), ('Kevin', 71), ('Kevin', 76), ('Kevin', 77), ('Laura', 126), ('Laura', 127), ('Laura', 138), ('Laura', 139), ('Michael', 105), ('Michael', 107), ('Michael', 115), ('Michael', 116), ('Norbert', 220), ('Norbert', 220), ('Norbert', 236), ('Norbert', 237), ('Oliver', 144), ('Oliver', 144), ('Oliver', 156), ('Oliver', 156), ('Patricia', 72), ('Patricia', 78),

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 300), ('Bob', 510), ('Charlie', 449), ('Dan', 535), ('Edith', 563), ('Frank', 850), ('George', 100), ('Hannah', 340), ('Ingrid', 333), ('Jerry', 549), ('Kevin', 294), ('Laura', 530), ('Michael', 443), ('Norbert', 913), ('Oliver', 600), ('Patricia', 300), ('Quinn', 498), ('Ray', 350), ('Sarah', 550), ('Tim', 446), ('Ursula', 484), ('Victor', 445), ('Wendy', 399), ('Xavier', 444), ('Yvonne', 432), ('Zelda', 550)]
CPU times: user 134 ms, sys: 19.5 ms, total: 153 ms
Wall time: 534 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Ingrid,"[{'transaction-id': 41, 'amount': -248}, {'tra..."
1,1,Dan,"[{'transaction-id': 140, 'amount': 11236}, {'t..."
2,2,Laura,"[{'transaction-id': 484, 'amount': -1620}, {'t..."
3,3,Frank,"[{'transaction-id': 45, 'amount': 539}, {'tran..."
4,4,Quinn,"[{'transaction-id': 11, 'amount': 76}, {'trans..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 220 ms, sys: 6.56 ms, total: 226 ms
Wall time: 1.62 s


name
Alice      300
Bob        510
Charlie    449
Dan        535
Edith      563
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Ingrid', 'amount': -248, 'transaction-id': 41},
 {'id': 0, 'name': 'Ingrid', 'amount': -192, 'transaction-id': 91},
 {'id': 0, 'name': 'Ingrid', 'amount': -213, 'transaction-id': 156})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Ingrid,-248,41
1,0,Ingrid,-192,91
2,0,Ingrid,-213,156
3,0,Ingrid,-10,209
4,0,Ingrid,-162,215


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 182 ms, sys: 18 ms, total: 200 ms
Wall time: 1.15 s


name
Alice        5356
Bob         22941
Charlie     15892
Dan         24139
Edith       20209
Frank       49660
George       1377
Hannah       9977
Ingrid      25868
Jerry       25664
Kevin       12550
Laura       33106
Michael     19168
Norbert     28744
Oliver      23612
Patricia    16151
Quinn       15902
Ray         16535
Sarah       21983
Tim         21316
Ursula      25008
Victor       8427
Wendy       13973
Xavier      16214
Yvonne       7904
Zelda       18324
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()